<font size="20">

# TEMPLATE NOTEBOOK

<font size="3">
    
> __Author:__ XXX
<br/>__Institute:__ XXX
<br/>__Date:__ XX/XX-XXXX

_Abstract:_
This is a template notebook used for multi-grid analysis. Please copy this notebook, rename it, and commit, before editing and pushing new content. Place your raw data in the '/data/template/raw/'-folder. The test-data in that folder can be deleted after initial testing.
    
    
_How to run the notebook:_ 
* Section 1.1 - Compile the cell to import dependencies
* Section 1.2 - Define global parameters, such as paths to data folder
* Section 1.3 - Compile all cells to define all necesarry functions

* Section 2.1 - Declare paths to all data sets
* Section 2.2 - Extract, cluster and save all data sets
* Section 2.3 - Load clusters and events into the Jupyter Notebook

* Section 3.  - Declare all filters data will be used on the data

* Section 4.  - Plot all data

# Contents
    
* [1. Introduction](#INTRODUCTION)
    * [1.1 Packages](#PACKAGES)
    * [1.2 Parameters](#PARAMETERS)
    * [1.3 Functions](#FUNCTIONS)
* [2. Data](#DATA)
    * [2.1 Paths](#PATHS)
    * [2.2 Extract](#EXTRACT)
    * [2.3 Load](#LOAD)
* [3. Filters](#FILTER)
* [4. Plotting](#PLOTTING)

# 1. Introduction

Summary of content in notebook.

## 1.1 Packages<a class="anchor" id="PACKAGES"></a>

Import all necessary packages.

In [ ]:
# Autoload packages when doing an external change
%load_ext autoreload
%autoreload 2

# General packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import plotly.graph_objects as go
from plotly.offline import iplot, plot

# Data handling
import mg.file_handling.read as mg_read
import mg_cncs.read as cncs_read

# Plotting
import mg_let.basic_plot as mg_basic_plot

# Helper functions
import mg.helper_functions.misc as mg_hf
import mg_cncs.helper_functions as cncs_hf

# Import widgets
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, HBox, Label

# For animation
import imageio
import shutil

## 1.2 Parameters

In [ ]:
# Declare all global parameters
RAW_FOLDER = '../data/template/raw/'
PROCESSED_FOLDER = '../data/template/processed/'

## 1.3 Global functions

Declare all functions used in the analysis.

### Extract

In [ ]:
def extract_mg_data(zipped_path, clusters_save_path, events_save_path):
    """
    Function to extract, cluster and save data.
    
    Args:
        zipped_path (str): Location of raw data
        clusters_save_path (str): Destination for clusters
        events_save_path (str): Destination for events

    Yields:
        Clusters and events are extracted from the raw data and saved at the specified locations
    
    """
    unzipped_path = mg_read.unzip_data(zipped_path)
    data = mg_read.import_data(unzipped_path)
    # Extract clusters and save to disc
    clusters = mg_read.extract_clusters(data)
    mg_read.save_data(clusters, clusters_save_path)
    clusters = None
    # Extract events and save to disc
    events = mg_read.extract_events(data)
    mg_read.save_data(events, events_save_path)
    events = None
    # Clear data
    data = None

In [ ]:
# Extract data, cluster, and save to file
def extract_and_save(run, raw_path):
    """
    Function to extract, cluster and save data.
    
    Args:
        run (str): File run, as specified in the 'Paths'-declaration below
        raw_path (str): Path to the raw data in the '.zip'-file

    Yields:
        Clusters and events are extracted from the raw data and saved in the 'processed'-folder.
    
    """
    clusters_path = PROCESSED_FOLDER + run + '_clu.h5'
    events_path = PROCESSED_FOLDER + run + '_ev.h5'
    extract_mg_data(raw_path, clusters_path, events_path)

### Load

In [ ]:
# Load clusters and events
def load_clusters_and_events(run):
    """
    Function to load data from a specific run.
    
    Args:
        run (str): File run, as specified in the 'Paths'-declaration below

    Returns:
        Clusters (DataFrame)
        Events (DataFrame)
    
    """
    clusters_path = PROCESSED_FOLDER + run + '_clu.h5'
    events_path = PROCESSED_FOLDER + run + '_ev.h5'
    return mg_read.load_data(clusters_path), mg_read.load_data(events_path)

### Plotting (SEQUOIA prototype)

In [ ]:
def plot_basic(run, clusters_unfiltered, events, df_filter, bus_start, bus_stop, number_rows, area):
    """
    Function to plot all basic plots for SEQUOIA detector,
    such as PHS, Coincidences and rate.
    
    Ordering of plotting is:
    
    PHS 2D - NOT FILTERED
    MULTIPLICITY - NOT FILTERED
    PHS 1D - FILTERED
    COINCIDENCES 2D - FILTERED
    PHS CORRELATION - FILTERED
    RATE - FILTERED
    TOF - FILTERED
    
    Note that all plots are filtered except the first two.
    
    Args:
        run (str): File run
        clusters_unfiltered (DataFrame): Unfiltered clusteres
        events (DataFrame): Individual events
        df_filter (dict): Dictionary specifying the filter which will be used on the clustered data
        bus_start (int): First bus to plot
        bus_stop (int): Last bus to plot
        number_rows (int): Number of rows in plots (number of rows must be larger than number_buses/3)
        area (float): Area in m^2 of the active detector surface

    Yields:
        Plots the basic analysis
    
    """
    mg_hf.set_thick_labels(12)
    
    # Filter clusters
    clusters = mg_read.filter_data(clusters_unfiltered, df_filter)
    
    # Declare parameters
    duration = (clusters_unfiltered.time.values[-1] - clusters_unfiltered.time.values[0]) * 62.5e-9
    
    # PHS - 2D
    vmin = 1
    vmax = events.shape[0] // 1000 + 100
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        events_bus = events[events.bus == bus]
        if events_bus.shape[0] > 0:
            mg_basic_plot.phs_2d_plot(events_bus, bus, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_phs_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Multiplicity
    vmin = None
    vmax = None
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 1:
            mg_basic_plot.multiplicity_plot_perc(clusters_bus, bus, duration)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_multiplicity_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # PHS - 1D
    vmin = None
    vmax = None
    bins_phs_1d = 300
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        clusters_uf_bus = clusters_unfiltered[clusters_unfiltered.bus == bus]
        mg_basic_plot.phs_clusters_1d_plot(clusters_bus, clusters_uf_bus, bins_phs_1d, bus, duration)
        plt.yscale('log')
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_phs_1d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - 2D
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    if clusters.shape[0] != 0:
        vmin = (1 * 1/duration)
        vmax = (clusters.shape[0] // 450 + 5) * 1/duration
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        # Calculate number of events and rate in a specific bus
        number_events = clusters_bus.shape[0]
        events_per_s = number_events/duration
        events_per_s_m2 = events_per_s/area
        title = ('Bus %d\n(%d events, %.6f events/s/m$^2$)' % (bus, number_events, events_per_s_m2))
        if number_events > 1:
            mg_basic_plot.clusters_2d_plot(clusters_bus, title, vmin, vmax, duration)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_coincidences_2d.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences - PHS
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    if clusters.shape[0] != 0:
        vmin = 1/duration
        vmax = (clusters.shape[0] // 450 + 1000) / duration
    else:
        duration = 1
        vmin = 1
        vmax = 1
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        if clusters_bus.shape[0] > 1:
            mg_basic_plot.clusters_phs_plot(clusters_bus, bus, duration, vmin, vmax)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_coincidences_phs.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    # Rate 
    number_bins = 50
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.rate_plot(clusters_bus, number_bins, bus, area)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_rate.png' % run
    fig.savefig(output_path, bbox_inches='tight')
    
    
    # TIME-OF-FLIGHT
    number_bins = 300
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(4*number_rows)
    for i, bus in enumerate(np.arange(bus_start, bus_stop+1, 1)):
        plt.subplot(number_rows, 3, i+1)
        clusters_bus = clusters[clusters.bus == bus]
        mg_basic_plot.tof_histogram(clusters_bus, number_bins, bus)
    plt.tight_layout()
    fig.show()
    
    # Save data
    output_path = '../output/%s_tof.png' % run
    fig.savefig(output_path, bbox_inches='tight')

### Plotting (CNCS prototype)

In [ ]:
def plot_basic_cncs(run, path, clusters_unfiltered, df_filter, area, extra=''):
    # Set plot label thicknesses
    mg_hf.set_thick_labels(15)
    
    # Filter clusters
    clusters = mg_read.filter_data(clusters_unfiltered, df_filter)
    
    # Declare parameters
    duration = cncs_hf.get_measurement_time(path)
    phs_attributes = ['w_adc_m1', 'w_adc_m2','g_adc_m1', 'g_adc_m2']
    phs_titles = ['Wires (multiplicity 1)', 'Wires (multiplicity 2)',
                  'Grids (multiplicity 1)', 'Grids (multiplicity 2)']
    
    # PHS 
    fig = plt.figure()
    fig.set_figwidth(12)
    fig.set_figheight(10)
    for i, (attribute, title) in enumerate(zip(phs_attributes, phs_titles)):
        plt.subplot(2, 2, i+1)
        plt.hist(clusters[attribute], histtype='step', color='black',
                 range=[0, 4095], bins=100, zorder=5, label='Filtered')
        plt.hist(clusters_unfiltered[attribute], histtype='step', color='red',
                 range=[0, 4095], bins=100, zorder=5, label='Unfiltered')
        plt.xlabel('Charge (adc channels)')
        plt.ylabel('Counts') 
        plt.title(title)
        plt.yscale('log')
        plt.grid(True, which='major', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.legend()
    plt.tight_layout()
    fig.show()
    output_path = '../output/%s_phs_plot%s.png' % (run, extra)
    fig.savefig(output_path, bbox_inches='tight')
    
    # Coincidences
    grids = clusters['g_ch_m1']
    wires = clusters['w_ch_m1']
    number_events = len(grids)
    vmin = 1
    vmax = (number_events // 450 + 5)
    
    fig = plt.figure()
    plt.suptitle('Coincidences')
    fig.set_figwidth(12)
    fig.set_figheight(5)
    plt.subplot(1, 2, 1)
    plt.hist2d(wires, grids,
               bins=[64, 48],
               range=[[-0.5, 63.5], [-0.5, 47.5]],
               vmin=vmin, vmax=vmax,
               norm=LogNorm(),
               cmap='jet')
    plt.xlabel('Wire (Channel number)')
    plt.ylabel('Grid (Channel number)')
    plt.title('Module 1')
    cbar = plt.colorbar()
    cbar.set_label('Counts')
    plt.subplot(1, 2, 2)
    plt.hist2d(wires, grids,
           bins=[64, 48],
           range=[[63.5, 127.5], [47.5, 95.5]],
           norm=LogNorm(),
           vmin=vmin, vmax=vmax,
           cmap='jet')
    plt.xlabel('Wire (Channel number)')
    plt.ylabel('Grid (Channel number)')
    plt.title('Module 2')
    cbar = plt.colorbar()
    cbar.set_label('Counts')
    plt.tight_layout()
    fig.show()
    output_path = '../output/%s_coincidences_plot%s.png' % (run, extra)
    fig.savefig(output_path, bbox_inches='tight')
    
    # Delimiters
    plot_delimiters(run + extra, clusters)
    
    # Coincidences full
    fig = plt.figure()
    fig.set_figwidth(12)
    fig.set_figheight(10)
    plt.hist2d(wires, grids,
               weights=np.ones(len(wires))*(1/duration),
               bins=[128, 96],
               range=[[-2, 127.5], [-2, 95.5]],
               norm=LogNorm(), cmap='jet',
               vmin=1e-5, vmax=5e-4
               )
    plt.xlabel('Wire (Channel number)')
    plt.ylabel('Grid (Channel number)')
    plt.title('Coincidences')
    cbar = plt.colorbar()
    cbar.set_label('Counts/s')
    fig.show()
    output_path = '../output/%s_coincidences_full_plot%s.png' % (run, extra)
    fig.savefig(output_path, bbox_inches='tight')
    
    # Plot tof
    fig = plt.figure()
    fig.set_figwidth(10)
    fig.set_figheight(5)
    plt.hist(clusters['tof'], histtype='step', color='black', bins=500)
    plt.xlabel('tof (adc channels)')
    plt.ylabel('Counts') 
    plt.yscale('log')
    plt.tight_layout()
    plt.grid(True, which='major', zorder=0)
    plt.grid(True, which='minor', linestyle='--', zorder=0)
    fig.show()
    output_path = '../output/%s_tof_plot%s.png' % (run, extra)
    fig.savefig(output_path, bbox_inches='tight')
    
    # Print key values
    print('Number of events: %.2f' % number_events)
    print('Duration: %.2f hours' % (duration/(60*60)))
    print('Area: %.2f m^2' % area)
    print('Rate: %.2f Hz/m^2' % (number_events/(duration*area)))

def plot_delimiters(run, clusters):
    # Import delimiters
    delimiters_dict = cncs_hf.import_delimiter_table()
    
    # Iterate through all attributes
    events = ['wires', 'wires', 'grids', 'grids']
    attributes = ['w_ch_adc_m1', 'w_ch_adc_m2', 'g_ch_adc_m1', 'g_ch_adc_m2']
    intervals = [16, 16, 96, 96]
    plot_ranges = [[0, 4000], [0, 4000], [0, 4000], [0, 4000]]
    for i, (event, attribute, interval, plot_range) in enumerate(zip(events, attributes, intervals, plot_ranges)):
        delimiters = delimiters_dict[event]
        fig = plt.figure()
        fig.set_figwidth(12)
        fig.set_figheight(5)
        plt.grid(True, which='major', zorder=0)
        plt.grid(True, which='minor', linestyle='--', zorder=0)
        plt.hist(clusters[attribute], histtype='step', color='black',
                 range=plot_range, bins=1500, zorder=10)
        plt.xlabel('Charge (adc channels)')
        plt.ylabel('Counts') 
        plt.title(attribute)
        plt.yscale('log')
        for delimiter in delimiters:
            plt.axvline(delimiter[0], color='red', zorder=15)
            plt.axvline(delimiter[1], color='red', zorder=15)
            small_delimiters = np.linspace(delimiter[0], delimiter[1], interval+1)
            previous_delimiter = small_delimiters[0]
            for delimiter in small_delimiters[1:]:
                plt.axvline(previous_delimiter, color='blue', zorder=5)
                plt.axvline(delimiter, color='blue', zorder=5)
                previous_delimiter = delimiter
                
        plt.tight_layout()
        fig.show()
    
        # Save data
        output_path = '../output/%s_delimiters_plot_%d.png' % (run, i)
        fig.savefig(output_path, bbox_inches='tight')

# 2. Data

## 2.1 Paths<a class="anchor" id="EXTRACT"></a>

In [ ]:
SEQUOIA_PATHS = {'sequoia_run_1': RAW_FOLDER + 'sequoia_test_1.zip',
                 'sequoia_run_2': RAW_FOLDER + 'sequoia_test_2.zip'}

CNCS_PATHS = {'cncs_run_1': RAW_FOLDER + 'cncs_test_1'}

## 2.2 Extract<a class="anchor" id="EXTRACT"></a>

In [ ]:
# Note that the 'extract_and_save'-function only needs to performed one.
# After that, the data has been clustered and saved. The data can then
# be accessed using the 'load_clusters_and_events'-function.

extract_and_save('sequoia_run_1', SEQUOIA_PATHS['sequoia_run_1'])
extract_and_save('sequoia_run_2', SEQUOIA_PATHS['sequoia_run_2'])

## 2.3 Load<a class="anchor" id="LOAD"></a>

In [ ]:
clu_sequoia_run_1, ev_sequoia_run_1 = load_clusters_and_events('sequoia_run_1')
clu_sequoia_run_2, ev_sequoia_run_2 = load_clusters_and_events('sequoia_run_2')

# 3. Filters<a class="anchor" id="FILTER"></a>

Declare filters which can be used on the data. Below are a few template filters.

## SEQUOIA

In [ ]:
"""
Filters are declared in the following format: {'PARAMETER': [MIN_VALUE, MAX_VALUE, IS_ACTIVATE]}
"""

# Declare filters for SEQUOIA module
mg_filter_basic = {'wm': [1, 1, True],                   # Wire multiplicity
                   'gm': [1, 5, True],                   # Grid multiplicity
                   'wadc': [1200, np.inf, True],         # Wire charge
                   'gadc': [1200, np.inf, True],         # Grid charge
                   'tof': [0, np.inf, True],             # Time-of-flight (TDC channels)
                   'time': [0, np.inf, True],            # Time (TDC channels)
                   'bus': [0, 8, True],                  # Bus
                   'flag': [0, 1, False],                # =1 if different buses within same coincidence
                   'layer': [0, 19, False],              # Layer, front=0 to back=19
                   'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                   'gch': [80, 119, True]}               # Grid channel, bottom=80 to top=119

mg_filter_no_edges = {'wm': [1, 1, True],                 
                      'gm': [1, 5, True],                  
                      'wadc': [1200, np.inf, True],        
                      'gadc': [1200, np.inf, True],          
                      'tof': [0, np.inf, True],            
                      'time': [0, np.inf, True],            
                      'bus': [0, 8, True],                 
                      'flag': [0, 1, False],                
                      'layer': [0, 19, False],              
                      'row': [0, 11, False],                
                      'gch': [82, 117, True]}               

mg_filter_no_edges_no_back = {'wm': [1, 1, True],                   
                              'gm': [1, 5, True],                  
                              'wadc': [1200, np.inf, True],          
                              'gadc': [1200, np.inf, True],          
                              'tof': [0, np.inf, True],             
                              'time': [0, np.inf, True],      
                              'bus': [0, 8, True],                  
                              'flag': [0, 1, False],                
                              'layer': [0, 15, True],              
                              'row': [0, 11, False],                
                              'gch': [82, 117, True]}               

mg_filter_no_edges_no_back_no_middle = {'wm': [1, 1, True],             
                                        'gm': [1, 5, True],                
                                        'wadc': [1200, np.inf, True],         
                                        'gadc': [1200, np.inf, True],          
                                        'tof': [0, np.inf, True],           
                                        'time': [0, np.inf, True],           
                                        'bus': [0, 8, True],                  
                                        'flag': [0, 1, False],               
                                        'layer': [0, 15, True],            
                                        'row': [0, 11, False],               
                                        'gch': [82, 95, True]}               

## CNCS

In [ ]:
# Declare filter for the CNCS module
mg_filter_cncs = {'w_adc_m1': [1500, np.inf, True],
                  'w_adc_m2': [0, np.inf, False],
                  'w_ch_adc_m1': [200, np.inf, True],
                  'w_ch_adc_m2': [0, np.inf, False],
                  'g_adc_m1': [1200, np.inf, True],
                  'g_adc_m2': [0, np.inf, False],
                  'g_ch_adc_m1': [200, np.inf, True],
                  'g_ch_adc_m2': [0, np.inf, False]}

mg_no_filter_cncs = {'w_adc_m1': [300, np.inf, False],
                     'w_adc_m2': [300, np.inf, False],
                     'w_ch_adc_m1': [300, np.inf, False],
                     'w_ch_adc_m2': [300, np.inf, False],
                     'g_adc_m1': [300, np.inf, False],
                     'g_adc_m2': [300, np.inf, False],
                     'g_ch_adc_m1': [300, np.inf, False],
                     'g_ch_adc_m2': [300, np.inf, False],
                     'tof': [0, np.inf, False],
                     'w_ch_m1': [0, np.inf, False],
                     'g_ch_m1': [0, np.inf, False],
                     'g_ch_m2': [0, np.inf, False]}

# 4. Plotting<a class="anchor" id="FILTER"></a>

## SEQUOIA

### sequoia_run_1

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('sequoia_run_1', clu_sequoia_run_1, ev_sequoia_run_1, mg_filter_basic, 0, 5, 2, area)

### sequoia_run_2

In [ ]:
area = 0.0225*0.0225*4*40
plot_basic('sequoia_run_2', clu_sequoia_run_2, ev_sequoia_run_2, mg_filter_basic, 0, 5, 2, area)

## CNCS

### cncs_run_1

In [ ]:
# Import data and declare parameters
run = 'cncs_run_1'
path = CNCS_PATHS['cncs_run_1']
clusters_unfiltered = cncs_read.extract_clusters(CNCS_PATHS['cncs_run_1'])
df_filter = mg_filter_cncs
area = 0.0225 * 0.0225 * 4 * 48 * 2 # m^2

# Plot data
plot_basic_cncs(run, path, clusters_unfiltered, df_filter, area, extra='_with_filter')